In [2]:
import pandas as pd


In [174]:
# Load the JSON file
df = pd.read_json("CIK0000320193.json")
# df = pd.read_json("CIK0000927628.json")
# df = pd.read_json("CIK0001415603.json")

In [175]:
yearLimit = 2023

In [176]:
# Print the DataFrame
df.head()

,cik,entityName,facts
dei,320193,Apple Inc.,{'EntityCommonStockSharesOutstanding': {'label...
us-gaap,320193,Apple Inc.,{'AccountsPayable': {'label': 'Accounts Payabl...


The JSON files come in two main partitions (need to find out exactly what they mean)

- DEI

- US-GAAP

General Structure of the JSON file:

- [Partition]
    - Heading Name
        - Label (more verbose heading)
        - Description
        - Units
            - Unit of Measurement
                - ...
                - Val
                - Form
                - Filed (date)
                - ...
    

## Read Partition

In [177]:
# Pull out the company's name and CIK
entityName = df.iloc[0,1]
entityCIK = str(df.iloc[0,0])
print('CIK: ' + entityCIK)
print('Company: ' + entityName)

for i in range(len(df)):
    partitionName = df.index[i]
    partitionName = partitionName.upper()
    partition = df.iloc[i,2]
    subGroups = partition.items()

    # Initialise a dataframe to hold the data
    columnNames = ['company', 'CIK', 'heading', 'units', 'value', 'fy', 'qtr', 'frame']
    masterDf = pd.DataFrame(columns=columnNames)

    # DEI is broken down into subgroups
    for group in subGroups:
        # Each subgroup has a heading, detailed description, and unit of measurement
        row = {}
        heading = group[0]
        row['company'] = entityName
        row['CIK'] = entityCIK
        row['heading'] = heading
        desc = group[1]['description']

        # The units component contains the actual measurement and the associated metadata (in the 'records' list)
        units = group[1]['units']
        for unit, records in units.items():
            row['units'] = unit

            # Filter out rows where the latest year of data is less than 2024
            maxYear = 0
            for record in records:
                if int(record['fy']) > maxYear:
                    maxYear = int(record['fy'])

            if maxYear < yearLimit:
                continue

            # We have a record for every filing date
            for record in records:
                if record['form'] == '10-Q' and 'frame' in record.keys():
                    row['fy'] = record['fy']
                    row['qtr'] = record['fp']
                    row['value'] = round(float(record['val']), 2)
                    row['frame'] = record['frame']

                    # Append the row to the main dataframe
                    nextIdx = len(masterDf)
                    masterDf.loc[nextIdx] = row

    masterDf = masterDf.sort_values(by = ['heading', 'frame'])
    masterDf.to_csv(f'fin tables/{entityName} {partitionName}.csv', index=False)
    
    

CIK: 320193
Company: Apple Inc.


In [160]:
df.index[0]

'dei'

In [155]:
"""masterDf = masterDf.sort_values(by = ['headingFull', 'frame'])
masterDf.to_csv('Apple Financials.csv', index=False)"""

In [146]:
"""masterDf = masterDf.sort_values(by = ['headingFull', 'frame'])
masterDf.to_csv('Capital One Financials.csv', index=False)"""